In [14]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import torch.nn as nn
from torchvision import transforms, models

class Food101Dataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.classes = sorted(os.listdir(data_folder))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = self._load_images()

    def _load_images(self):
        images = []
        for cls_name in self.classes:
            cls_path = os.path.join(self.data_folder, cls_name)
            for img_name in os.listdir(cls_path):
                img_path = os.path.join(cls_path, img_name)
                images.append((img_path, self.class_to_idx[cls_name]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformation for the images
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Path to your data folder
data_folder = "C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/google images"

# Create dataset
food101_dataset = Food101Dataset(data_folder, transform=transform)

# Create dataloader
batch_size = 32
data_loader = DataLoader(food101_dataset, batch_size=batch_size, shuffle=True)

# for i in food101_dataset:
#     if i[1] == 0:
#         print(i)


# # Iterate through the dataloader for testing
# for images, labels in data_loader:
#     # Here you can feed the images to your model for testing
#     # Replace this with your model inference code
#     print(images[0].shape)
#     print(labels)
#     break  # Stop iteration after the first batch for testing purposes


# see_examples = 5
# for i, (imgs, label) in enumerate(data_loader):
#     clear_output(wait=True)
#     image = imgs[0].numpy().transpose(1, 2, 0)
#     plt.imshow(image)
#     plt.text(5, 15, label[0], fontsize ='xx-large', color='red', fontweight='bold')
#     plt.show()

#     if i >= see_examples - 1:
#       break
#     time.sleep(1)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(0))

def test_acc(net: nn.Module, test_loader: DataLoader):

  net.to(device)
  net.eval()
  
  total = 0
  correct = 0
  top2_correct = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()

    _, predicted2 = torch.topk(outputs, 2, dim=1)  # Get top 2 predictions
    top2_correct += ((predicted2 == labels.view(-1, 1)) | (predicted2[:, 1].view(-1, 1) == labels.view(-1, 1))).sum().item()

  return (correct / total * 100, top2_correct / total * 100)

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [16]:
class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, 101)
        )

    def forward(self, x):
        return self.classifier(x)

num_classes = len(food101_dataset.classes)
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])

for param in resnet.parameters():
    param.requires_grad = False

###

model = nn.Sequential(
    resnet,
    CustomClassifier()
)

In [17]:
start = time.time()
for i in range(1, 16):

    state_dict = torch.load(f'resnet_dropout/resnet50_multiplefclayers_adamw_epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy, top2accuracy = test_acc(model, data_loader)

    print(f'Epoch {i}: accuracy - {accuracy:.3f}%, top 2 accuracy - {top2accuracy:.3f}%')

print(f'Testing duration: {time.time() - start}')

Epoch 1: accuracy - 47.876%, top 2 accuracy - 75.503%
Epoch 2: accuracy - 53.222%, top 2 accuracy - 81.152%
Epoch 3: accuracy - 56.142%, top 2 accuracy - 82.924%
Epoch 4: accuracy - 57.672%, top 2 accuracy - 85.038%
Epoch 5: accuracy - 58.296%, top 2 accuracy - 83.850%
Epoch 6: accuracy - 58.931%, top 2 accuracy - 84.807%
Epoch 7: accuracy - 59.565%, top 2 accuracy - 84.555%
Epoch 8: accuracy - 59.686%, top 2 accuracy - 86.992%
Epoch 9: accuracy - 59.545%, top 2 accuracy - 86.327%
Epoch 10: accuracy - 59.394%, top 2 accuracy - 84.041%
Epoch 11: accuracy - 57.672%, top 2 accuracy - 82.662%


KeyboardInterrupt: 